**Algorithms and Applications in Social Networks - HW #2**

In [33]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np

**Question #1**

a. Implement Newman-Girvan algorithm for non-overlapping communities. The
algorithm should receive a network and parameter k (number of communities)
and return the communities.

In [34]:
def newman_girvan(G, k):
  communities = []
  for connected_component in nx.connected_components(G):
    communities.append(list(connected_component))
  while(G.number_of_edges() > 0 and len(communities) != k):
    betweenness_of_all_edges = nx.edge_betweenness_centrality(G)
    max_edge = max(betweenness_of_all_edges, key=betweenness_of_all_edges.get)
    G.remove_edge(*max_edge)
    communities = []
    for connected_component in nx.connected_components(G):
      communities.append(list(connected_component))
  return communities

b. Run this algorithm on the biggest connected component of the following
dataset: https://bit.ly/2KLHN60 (with k=3).
Each line of the file represents an edge between two nodes.

In [ ]:
from urllib.request import Request, urlopen
#get the data from the file
url = "https://bit.ly/2KLHN60"
request = Request(url)
request.add_header('user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
data = urlopen(request)
#crate graph from the data
G=nx.Graph()
for line in data:
  edge = str(line,'utf-8').split()
  G.add_edge(edge[0], edge[1])
#find the biggest connected component
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
biggest_connected_component = nx.Graph(G.subgraph(Gcc[0]))
#run newman_girvan on the biggest connected component
communities = newman_girvan(biggest_connected_component, k=3)
for index in range(len(communities)):
  print("community", index + 1, ": ", communities[index])

**Question #2**

a. Implement k-clique communities detection algorithm. The algorithm should
receive a network and parameter k and return the communities.

In [36]:
def k_clique_communities_detection(G, k):
  #find all maximal cliques
  maximal_cliques = list(nx.find_cliques(G))
  number_of_maximal_cliques = len(maximal_cliques)
  #create clique overlap matrix
  clique_overlap_matrix = np.empty((number_of_maximal_cliques, number_of_maximal_cliques), dtype=int)
  for row in range(number_of_maximal_cliques):
    for col in range(number_of_maximal_cliques):
      clique_overlap_matrix[row][col] = len(np.intersect1d(maximal_cliques[row], maximal_cliques[col]))
  #threshold matrix with k-1
  for row in range(number_of_maximal_cliques):
    for col in range(number_of_maximal_cliques):
      if (row != col and clique_overlap_matrix[row][col] >= k-1) or (row == col and clique_overlap_matrix[row][col] >= k):
        clique_overlap_matrix[row][col] = 1
      else:
        clique_overlap_matrix[row][col] = 0
  #communities are connected components
  G_communities = nx.Graph()
  G_communities.add_nodes_from(range(number_of_maximal_cliques))
  for row in range(number_of_maximal_cliques):
    for col in range(number_of_maximal_cliques):
      if row >= col and clique_overlap_matrix[row][col] == 1:
        G_communities.add_edge(row, col)
  zero_degree_nodes = [node for node,degree in dict(G_communities.degree()).items() if degree == 0]
  G_communities.remove_nodes_from(zero_degree_nodes)
  connected_components = list(nx.connected_components(G_communities))
  communities = []
  for connected_component in connected_components:
    community = np.empty(0)
    for node in connected_component:
      community = np.unique(np.concatenate((community, maximal_cliques[node]), axis=0))
    communities.append(community)
  return communities

b. Run this algorithm on the biggest connected component of the following
dataset: https://bit.ly/2KLHN60 (with k=4).


In [ ]:
from urllib.request import Request, urlopen
#get the data from the file
url = "https://bit.ly/2KLHN60"
request = Request(url)
request.add_header('user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
data = urlopen(request)
#crate graph from the data
G=nx.Graph()
for line in data:
  edge = str(line,'utf-8').split()
  G.add_edge(edge[0], edge[1])
#find the biggest connected component
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
biggest_connected_component = nx.Graph(G.subgraph(Gcc[0]))
#run k-clique communities detection on the biggest connected component
communities = k_clique_communities_detection(biggest_connected_component, k=4)
for index in range(len(communities)):
  print("community", index + 1, ": ", communities[index])